<a href="https://colab.research.google.com/github/Akashkunwar/dscodes/blob/master/cases_vaccination.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
###COVID DATA
## Retreving Data
# Assigning Github URLs
cases_url="https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
deaths_url="https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
recovered_url="https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"

# Reading URLs through Pandas liberary
total_cases = pd.read_csv(cases_url)
total_deaths = pd.read_csv(deaths_url)
total_recovered = pd.read_csv(recovered_url)

## Cleaning Data
# Extreacting and storing column dates in dates
dates = total_cases.columns[4:]

# Reshaping total cases column from wide shape to df shape
total_cases_df = total_cases.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Confirmed'
    )

# Reshaping total deaths column from wide shape to df shape
total_deaths_df = total_deaths.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Deaths'
    )

# Reshaping total recovered column from wide shape to df shape
total_recovered_df = total_recovered.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Recovered'
)

# Merging total_cases_df and total_deaths_df
covid_df = total_cases_df.merge(
  right=total_deaths_df, 
  how='left',
  on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long']
)

# Merging covid_df and total_recovered_df
covid_df = covid_df.merge(
  right=total_recovered_df, 
  how='left',
  on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long']
)

# Converting Date to proper datetime format
covid_df["Date"] = pd.to_datetime(covid_df["Date"])

# Inserting converted date to full table
covid_df = covid_df.sort_values(['Country/Region', 'Date'])
#covid_df = covid_df.sort_values(['Country/Region', 'Date'], ascending=[True, True])
# Reseting Index
# covid_df = covid_df.reset_index()

# Copying covid_df to covid_df
#covid_df = covid_df.copy()

# Renaming column names
covid_df = covid_df.rename(columns={'Country/Region': 'Country',
                                    'Province/State':'Province',
                                    'Confirmed': 'Total cases',
                                    'Deaths': 'Total deaths',
                                    'Recovered':'Total recovered'})

# Filling NaN value of provinve with country name and storing it in coiuntry column
covid_df["Province"] = covid_df["Province"].fillna(covid_df["Country"])
covid_df["Country"] = covid_df["Province"]

# Adding New cases, New deaths, New recovered to covid_df
covid_df["New cases"] = covid_df.groupby('Country')['Total cases'].diff().fillna(0)
covid_df["New deaths"] = covid_df.groupby('Country')['Total deaths'].diff().fillna(0)
covid_df["New recovered"] = covid_df.groupby('Country')['Total recovered'].diff().fillna(0)

# Converting float value to int in covid_df columns
covid_df["Total recovered"] = covid_df["Total recovered"].fillna(0)
covid_df["Total recovered"] = covid_df["Total recovered"].astype(int).fillna(0)
covid_df["New cases"] = covid_df["New cases"].astype(int)
covid_df["New deaths"] = covid_df["New deaths"].astype(int)
covid_df["New recovered"] = covid_df["New recovered"].astype(int)

# Arranging Columns
covid_df = covid_df[['Date', 'Province', 'Country','Lat','Long',
 'Total cases', 'Total deaths','Total recovered', 'New cases','New deaths','New recovered']]

# Removind unnecessary columns
covid_df = covid_df.drop(['Province', 'Lat', 'Long', 'Total recovered', 'New recovered'],axis=1)

# Reseting jumbled index in order
covid_df = covid_df.reset_index(drop=True)

In [ ]:
###VACCINE DATA
## Retreving Data
# Assigning Github URLs
vaccine_data = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.csv')
vaccine_data['date']=pd.to_datetime(vaccine_data.date)

#Date format change
vaccine_data['date'] = pd.to_datetime(vaccine_data['date'],format='%y-%m-%d').dt.date

#Dropping uncecessary data
vaccine_data.drop(['total_vaccinations_per_hundred',
                   'iso_code',
                   'people_vaccinated_per_hundred',
                   'people_fully_vaccinated_per_hundred',
                   'daily_vaccinations_per_million',
                   'daily_vaccinations_raw',
                   'total_vaccinations',
                   'people_vaccinated',
                   'daily_people_vaccinated_per_hundred',
                   'total_boosters_per_hundred'], inplace=True, axis=1)

#Rename vaccines to vaccine_name
# vaccine_data.rename(columns = {'vaccines':'vaccine_name'}, inplace = True) 

#Adding total vaccination column
Total_vaccinations = vaccine_data.groupby('location')['daily_vaccinations'].cumsum()
vaccine_data.insert(5,'Total_vaccinations',Total_vaccinations)

#Renaming columns
vaccine_df = vaccine_data.rename(columns={'location': 'Country',
                                    'date':'Date',
                                    'people_fully_vaccinated': 'Fully vaccinated',
                                    'total_boosters': 'Total boosters',
                                    'Total_vaccinations':'Total vaccinations',
                                    'daily_vaccinations': 'Daily vaccinations',
                                    'daily_people_vaccinated': 'Daily people vaccinated'})
#Arranging columns
vaccine_df = vaccine_df[['Date',
                         'Country',
                         "Fully vaccinated",
                         'Total vaccinations',
                         'Total boosters',
                         'Daily vaccinations',
                         'Daily people vaccinated']]

# Converting Date to proper datetime format
vaccine_df["Date"] = pd.to_datetime(vaccine_df["Date"])
vaccine_df = vaccine_df.fillna(0)
vaccine_df = vaccine_df.astype({"Fully vaccinated":'int',
                                'Total vaccinations':'int',
                                'Total boosters':'int',
                                'Daily vaccinations':'int',
                                'Daily people vaccinated':'int'})

#Removing unnecessary rows
# vaccine_df = vaccine_df.set_index("Country").drop("World")
# vaccine_df.reset_index(inplace=True)

In [ ]:
covid_df

,Date,Country,Total cases,Total deaths,New cases,New deaths
0,2020-01-22,Afghanistan,0,0,0,0
1,2020-01-23,Afghanistan,0,0,0,0
2,2020-01-24,Afghanistan,0,0,0,0
3,2020-01-25,Afghanistan,0,0,0,0
4,2020-01-26,Afghanistan,0,0,0,0
...,...,...,...,...,...,...
229183,2022-04-03,Zimbabwe,246525,5446,44,0
229184,2022-04-04,Zimbabwe,246612,5451,87,5
229185,2022-04-05,Zimbabwe,246744,5451,132,0
229186,2022-04-06,Zimbabwe,246803,5453,59,2


In [ ]:
vaccine_df

,Date,Country,Fully vaccinated,Total vaccinations,Total boosters,Daily vaccinations,Daily people vaccinated
0,2021-02-22,Afghanistan,0,0,0,0,0
1,2021-02-23,Afghanistan,0,1367,0,1367,1367
2,2021-02-24,Afghanistan,0,2734,0,1367,1367
3,2021-02-25,Afghanistan,0,4101,0,1367,1367
4,2021-02-26,Afghanistan,0,5468,0,1367,1367
...,...,...,...,...,...,...,...
93942,2022-04-03,Zimbabwe,3561400,9333169,615286,121347,85662
93943,2022-04-04,Zimbabwe,3570255,9452575,637510,119406,83858
93944,2022-04-05,Zimbabwe,3572995,9562261,645527,109686,76556
93945,2022-04-06,Zimbabwe,3578170,9656159,665706,93898,62943


In [ ]:
# pd.concat([covid_df, vaccine_df], axis=1, join="inner")
pd.concat([covid_df, vaccine_df], axis=1)

,Date,Country,Total cases,Total deaths,New cases,New deaths,Date,Country,Fully vaccinated,Total vaccinations,Total boosters,Daily vaccinations,Daily people vaccinated
0,2020-01-22,Afghanistan,0,0,0,0,2021-02-22,Afghanistan,0.0,0.0,0.0,0.0,0.0
1,2020-01-23,Afghanistan,0,0,0,0,2021-02-23,Afghanistan,0.0,1367.0,0.0,1367.0,1367.0
2,2020-01-24,Afghanistan,0,0,0,0,2021-02-24,Afghanistan,0.0,2734.0,0.0,1367.0,1367.0
3,2020-01-25,Afghanistan,0,0,0,0,2021-02-25,Afghanistan,0.0,4101.0,0.0,1367.0,1367.0
4,2020-01-26,Afghanistan,0,0,0,0,2021-02-26,Afghanistan,0.0,5468.0,0.0,1367.0,1367.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
229183,2022-04-03,Zimbabwe,246525,5446,44,0,NaT,NaN,NaN,NaN,NaN,NaN,NaN
229184,2022-04-04,Zimbabwe,246612,5451,87,5,NaT,NaN,NaN,NaN,NaN,NaN,NaN
229185,2022-04-05,Zimbabwe,246744,5451,132,0,NaT,NaN,NaN,NaN,NaN,NaN,NaN
229186,2022-04-06,Zimbabwe,246803,5453,59,2,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# merged = covid_df.combine_first(vaccine_df)
# merged = merged.fillna(0)
# merged = merged.astype({"Fully vaccinated":'int',
#                                 'Total vaccinations':'int',
#                                 'Total boosters':'int',
#                                 'Daily vaccinations':'int',
#                                 'Daily people vaccinated':'int'})
# merged = merged[['Date',
#                  'Country',
#                  'New cases',
#                  'New deaths',
#                  'New recovered',
#                  'Total boosters',
#                  'Total cases',
#                  'Total deaths',
#                  'Total recovered',
#                  'Daily people vaccinated',
#                  'Daily vaccinations',
#                  'Fully vaccinated',
#                  'Total vaccinations']]
# merged
covid_df.combine_first(vaccine_df)

,Country,Daily people vaccinated,Daily vaccinations,Date,Fully vaccinated,New cases,New deaths,Total boosters,Total cases,Total deaths,Total vaccinations
0,Afghanistan,0.0,0.0,2020-01-22,0.0,0,0,0.0,0,0,0.0
1,Afghanistan,1367.0,1367.0,2020-01-23,0.0,0,0,0.0,0,0,1367.0
2,Afghanistan,1367.0,1367.0,2020-01-24,0.0,0,0,0.0,0,0,2734.0
3,Afghanistan,1367.0,1367.0,2020-01-25,0.0,0,0,0.0,0,0,4101.0
4,Afghanistan,1367.0,1367.0,2020-01-26,0.0,0,0,0.0,0,0,5468.0
...,...,...,...,...,...,...,...,...,...,...,...
229183,Zimbabwe,NaN,NaN,2022-04-03,NaN,44,0,NaN,246525,5446,NaN
229184,Zimbabwe,NaN,NaN,2022-04-04,NaN,87,5,NaN,246612,5451,NaN
229185,Zimbabwe,NaN,NaN,2022-04-05,NaN,132,0,NaN,246744,5451,NaN
229186,Zimbabwe,NaN,NaN,2022-04-06,NaN,59,2,NaN,246803,5453,NaN


In [ ]:
pd.merge(covid_df, vaccine_df, how="outer", on=["Date", "Country"])

,Date,Country,Total cases,Total deaths,New cases,New deaths,Fully vaccinated,Total vaccinations,Total boosters,Daily vaccinations,Daily people vaccinated
0,2020-01-22,Afghanistan,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
1,2020-01-23,Afghanistan,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,2020-01-24,Afghanistan,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,2020-01-25,Afghanistan,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,2020-01-26,Afghanistan,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
243934,2022-04-02,Wales,NaN,NaN,NaN,NaN,2401969.0,6867939.0,2003997.0,2035.0,686.0
243935,2022-04-03,Wales,NaN,NaN,NaN,NaN,2402290.0,6869966.0,2004322.0,2027.0,673.0
243936,2022-04-04,Wales,NaN,NaN,NaN,NaN,2402616.0,6871879.0,2004999.0,1913.0,650.0
243937,2022-04-05,Wales,NaN,NaN,NaN,NaN,2402919.0,6873837.0,2005772.0,1958.0,717.0


In [ ]:
pd.concat([covid_df, vaccine_df], axis=0, ignore_index=True)

,Date,Country,Total cases,Total deaths,New cases,New deaths,Fully vaccinated,Total vaccinations,Total boosters,Daily vaccinations,Daily people vaccinated
0,2020-01-22,Afghanistan,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
1,2020-01-23,Afghanistan,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,2020-01-24,Afghanistan,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,2020-01-25,Afghanistan,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,2020-01-26,Afghanistan,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
322638,2022-04-03,Zimbabwe,NaN,NaN,NaN,NaN,3561400.0,9333169.0,615286.0,121347.0,85662.0
322639,2022-04-04,Zimbabwe,NaN,NaN,NaN,NaN,3570255.0,9452575.0,637510.0,119406.0,83858.0
322640,2022-04-05,Zimbabwe,NaN,NaN,NaN,NaN,3572995.0,9562261.0,645527.0,109686.0,76556.0
322641,2022-04-06,Zimbabwe,NaN,NaN,NaN,NaN,3578170.0,9656159.0,665706.0,93898.0,62943.0


In [ ]:
pd.merge(covid_df, vaccine_df, how="outer", on=['Date','Country'])

,Date,Country,Total cases,Total deaths,New cases,New deaths,Fully vaccinated,Total vaccinations,Total boosters,Daily vaccinations,Daily people vaccinated
0,2020-01-22,Afghanistan,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
1,2020-01-23,Afghanistan,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,2020-01-24,Afghanistan,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,2020-01-25,Afghanistan,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,2020-01-26,Afghanistan,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
243934,2022-04-02,Wales,NaN,NaN,NaN,NaN,2401969.0,6867939.0,2003997.0,2035.0,686.0
243935,2022-04-03,Wales,NaN,NaN,NaN,NaN,2402290.0,6869966.0,2004322.0,2027.0,673.0
243936,2022-04-04,Wales,NaN,NaN,NaN,NaN,2402616.0,6871879.0,2004999.0,1913.0,650.0
243937,2022-04-05,Wales,NaN,NaN,NaN,NaN,2402919.0,6873837.0,2005772.0,1958.0,717.0


In [ ]:
vaccine_df

,Country,Date,Fully vaccinated,Total vaccinations,Total boosters,Daily vaccinations,Daily people vaccinated
0,Afghanistan,2021-02-22,0,0,0,0,0
1,Afghanistan,2021-02-23,0,1367,0,1367,1367
2,Afghanistan,2021-02-24,0,2734,0,1367,1367
3,Afghanistan,2021-02-25,0,4101,0,1367,1367
4,Afghanistan,2021-02-26,0,5468,0,1367,1367
...,...,...,...,...,...,...,...
93450,Zimbabwe,2022-04-03,3561400,9333169,615286,121347,85662
93451,Zimbabwe,2022-04-04,3570255,9452575,637510,119406,83858
93452,Zimbabwe,2022-04-05,3572995,9562261,645527,109686,76556
93453,Zimbabwe,2022-04-06,3578170,9656159,665706,93898,62943


In [ ]:
len(set(list(covid_df.Country.unique())) ^ set(list(vaccine_df.Country.unique())))

130

In [ ]:
list(covid_df.Country.unique())
list(vaccine_df.Country.unique())
len(list(set(list(covid_df.Country.unique())).intersection(list(vaccine_df.Country.unique()))))

In [ ]:
set(list(covid_df.Country.unique())) ^ set(list(vaccine_df.Country.unique()))

{'Africa',
 'Alberta',
 'Anhui',
 'Antarctica',
 'Asia',
 'Australia',
 'Australian Capital Territory',
 'Beijing',
 'Bonaire Sint Eustatius and Saba',
 'Bonaire, Sint Eustatius and Saba',
 'British Columbia',
 'Burma',
 'Cabo Verde',
 'Canada',
 'Cape Verde',
 'Channel Islands',
 'China',
 'Chongqing',
 'Congo',
 'Congo (Brazzaville)',
 'Congo (Kinshasa)',
 'Democratic Republic of Congo',
 'Diamond Princess',
 'England',
 'Eritrea',
 'Europe',
 'European Union',
 'Faeroe Islands',
 'Falkland Islands',
 'Falkland Islands (Malvinas)',
 'Faroe Islands',
 'French Guiana',
 'Fujian',
 'Gansu',
 'Grand Princess',
 'Guadeloupe',
 'Guangdong',
 'Guangxi',
 'Guizhou',
 'Hainan',
 'Hebei',
 'Heilongjiang',
 'Henan',
 'High income',
 'Holy See',
 'Hubei',
 'Hunan',
 'Inner Mongolia',
 'Jiangsu',
 'Jiangxi',
 'Jilin',
 'Korea, South',
 'Liaoning',
 'Low income',
 'Lower middle income',
 'MS Zaandam',
 'Macao',
 'Macau',
 'Manitoba',
 'Marshall Islands',
 'Martinique',
 'Mayotte',
 'Micronesia',
 